In [ ]:
### Desa Bolger and Emma Beyer
### Adv. GIS Final Project
### Fall 2024 Python Script

#The goal of this document is to process the data into something that will be workable in other python formats and ArcGIS.
#I want to get everything downloaded and in WGS1984!

In [ ]:
#Read in the packages
import pandas as pd
import geopandas as gpd
from shapely.geometry import Point
import matplotlib.pyplot as plt

In [ ]:
#DOWNLOAD WRECK/OBSTRUCTION DATA

#Read the CSV into a Pandas dataframe
WrecksFile = gpd.read_file('../Data/Raw/Wrecks_and_Obstructions.shp')
WrecksFile.head()

#Get Coordinate System
print("Current CRS:", WrecksFile.crs)
WrecksFile = WrecksFile.to_crs("EPSG:4326")

#Save file with the correct WGS1984
WrecksFile.to_file("../Data/Processed/Wrecks_wgs1984.shp")
print("Current CRS:", WrecksFile.crs) #good

#Reveal the columns
WrecksFile.columns

In [ ]:
#PLOT WRECKS DATA for my own visualization 
WrecksFile.plot(column = 'objectcate');

In [ ]:
#DOWNLOAD EEZ DATA

#Read the CSV into a Pandas dataframe
EEZFile = gpd.read_file('../Data/Raw/eez.shp')
EEZFile.head()

#Get Coordinate System
print("Current CRS:", EEZFile.crs)
EEZFile = EEZFile.to_crs("EPSG:4326")

#Save file with the correct WGS1984
EEZFile.to_file("../Data/Processed/EEZ_wgs1984.shp")
print("Current CRS:", EEZFile.crs) #good

#Reveal the columns
EEZFile.columns

In [ ]:
#PLOT EEZ DATA for my own visualization
EEZFile.plot(column = 'territory1');

In [ ]:
#DOWNLOAD NC BOUNDARY DATA

#Read the CSV into a Pandas dataframe
NCStateFile = gpd.read_file('../Data/Raw/NCcounty.shp')
NCStateFile.head()

#Get Coordinate System
print("Current CRS:", NCStateFile.crs) #good

#Save file with the correct WGS1984
NCStateFile.to_file("../Data/Processed/NC_wgs1984.shp")

#Reveal the columns
NCStateFile.columns

In [ ]:
#PLOT NC DATA for my own visualization

#Separate by land, state water, boundary water
NCStateWater = NCStateFile.dissolve(
    'WATER')

#Visualize data organization
NCStateWater.head

#Plot (for my own visualization) the NC land boundary
water_type_LAND = NCStateFile[NCStateFile['WATER'] == 'A']
water_type_LAND.plot()

#Save LandFile
water_type_LAND.to_file("../Data/Processed/NorthCarolinaLand.shp")


#Plot (for my own visualization) the NC boundary water
water_type_O = NCStateFile[NCStateFile['WATER'] == 'O']
water_type_O.plot()

#Plot (for my own visualization) the NC inland water
water_type_S = NCStateFile[NCStateFile['WATER'] == 'S']
water_type_S.plot()

In [ ]:
#check coordinate systems
NCStateFile.crs == EEZFile.crs #True
WrecksFile.crs == EEZFile.crs #True
WrecksFile.crs == NCStateFile.crs #True

#All shapefiles are now in DATA/PROCCESED FOLDER & are WGS1984.

In [ ]:
#Make a 200 nautical mile buffer around NC

#add in WGS1984 area
NCland = gpd.read_file("../Data/Processed/NorthCarolinaLand.shp")
NCland.head()

#Dissolve to one land mass!
NClandDissolved = NCland.dissolve(by="WATER") #they all have the same category for water so they will group together!

#Get Coordinate System
print("Current CRS:", NClandDissolved.crs) #good 

#WGS1984 will not be accurate for distance, need to get in equal projection
NClandDissolved = NClandDissolved.to_crs("EPSG:26918")

#get nautical miles in meters
buffer_distance = 200 * 1852  # in meters

# make my buffer!!!
NClandDissolved["buffer_200nm"] = NClandDissolved.geometry.buffer(buffer_distance)

#reproject to WGS1984, just want the 200 buffer layer.
NClandDissolved = NClandDissolved.set_geometry("buffer_200nm").to_crs("EPSG:4326")

#visualize to confirm it worked
ax = NClandDissolved.plot(color = "blue")
NCland.plot(ax = ax, color = "black")
plt.show() #ok that makes a buffer for sure!

#I had two geometry columns - can only have 1!
NClandDissolved = NClandDissolved.drop(columns=["geometry"])

#save buffer for other uses in ArcGIS
NClandDissolved.to_file("../Data/Processed/NCBuffer.shp")